In [39]:
from pyscf import gto, scf, mp, cc
a = 2 # bond length in a cluster
d = 2 # distance between each cluster
unit = 'b' # unit of length
na = 2  # size of a cluster (monomer)
nc = 1 # set as integer multiple of monomers
spin = 2 # spin per monomer
frozen = 2 # frozen orbital per monomer
elmt = 'O'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis=basis, spin=0, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mf.stability()

nfrozen = nc*frozen

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Tue Dec 23 17:14:32 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00000000000

In [40]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

lno_type = ['1h'] * 2
oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 5.9562434609434  delta_f= 5.95624  |g|= 0.000804366  3 KF 10 Hx
macro= 2  f(x)= 5.956250400138  delta_f= 6.93919e-06  |g|= 0.0010574  1 KF 2 Hx
macro= 3  f(x)= 5.9562504218726  delta_f= 2.17346e-08  |g|= 4.56513e-05  1 KF 2 Hx
macro X = 3  f(x)= 5.9562504218726  |g|= 4.56513e-05  6 intor 5 KF 14 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-1

In [44]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd

mlno = ulnoccsd.ULNOCCSD(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
thresh = 1e-6
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h'] if lno_type is None else lno_type
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
frag_res = [None] * nfrag
for ifrag,loidx in enumerate(frag_lolist):
    if ifrag == 0:
        if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
            orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
            lno_param = [
                [
                    {
                        'thresh': (
                            lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                            else lno_thresh[i]
                        ),
                        'pct_occ': (
                            lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                            else lno_pct_occ[i]
                        ),
                        'norb': (
                            lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                            else lno_norb[ifrag][i]
                        ),
                    } for i in [0, 1]
                ] for s in range(2)
            ]

        else:
            orbloc = lo_coeff[:,loidx]
            lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                            'norb': lno_norb[ifrag][i]} for i in [0,1]]

        lno_coeff, frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
        # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)

        # mo_occ = mlno.mo_occ
        # frozen, maskact = ulnoccsd.get_maskact(frozen, [mo_occ[0].size, mo_occ[1].size])
        # mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=frozen).set(verbose=mlno.verbose_imp)
        # mcc._s1e = mlno._s1e
        # mcc._h1e = mlno._h1e
        # mcc._vhf = mlno._vhf
        # if mlno.kwargs_imp is not None:
        #     mcc = mcc.set(**mlno.kwargs_imp)

        # frag_res[ifrag], t1, t2 =\
        #     ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
    else: break

LO occ proj: 1 active | 0 standby | 5 orthogonal
LO occ proj: 0 active | 0 standby | 6 orthogonal


In [45]:
# frozen = [[],[]]
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)
print(nfrzocc_a,nactocc_a,nactvir_a,nactorb_a)
print(nfrzocc_b,nactocc_b,nactvir_b,nactorb_b)

7 1 1 2
7 1 1 2


In [46]:
ncas = (nactorb_a,nactorb_b)
ncore = (nfrzocc_a,nfrzocc_b)
lno_act_a = lno_coeff[0][:,ncore[0]:ncore[0]+ncas[0]]
lno_act_b = lno_coeff[1][:,ncore[1]:ncore[1]+ncas[1]]
# lno_act = np.hstack([lno_act_a,lno_act_b])
print(lno_act_a.shape)
print(lno_act_b.shape)

(10, 2)
(10, 2)


In [47]:
s1e = mf.get_ovlp()
lno_act_aa = lno_coeff[0].T @ s1e @ lno_act_a
lno_act_ba = lno_coeff[0].T @ s1e @ lno_act_b
print(lno_act_aa)
print(lno_act_ba)

[[-4.77686072e-17  2.58592048e-16]
 [-1.34691487e-18 -2.75616456e-18]
 [ 2.02614516e-17 -3.78550936e-18]
 [-1.23189031e-16 -2.22700490e-16]
 [ 9.29508281e-17  1.87330171e-16]
 [ 3.16198801e-17  4.76863375e-17]
 [-2.92225020e-16  1.22960934e-17]
 [ 1.00000000e+00 -1.52294825e-16]
 [-1.56907641e-16  1.00000000e+00]
 [-1.03153856e-17  2.44619151e-16]]
[[ 3.60122494e-03 -6.40155008e-05]
 [ 1.77132235e-04 -1.79808579e-04]
 [ 3.15904921e-06 -1.20146821e-05]
 [-9.15938319e-02  2.42190165e-01]
 [ 7.18807414e-02 -1.89941850e-01]
 [ 2.54671553e-01 -8.53417064e-01]
 [ 1.54326977e-02 -5.17022036e-02]
 [ 9.46019198e-01  3.15420095e-01]
 [-1.62427331e-01  2.73463601e-01]
 [ 9.59021625e-09 -2.94559289e-08]]


In [48]:
def prjmo(prj,s1e,mo):
    # prj and reconstruct mo
    mo_rec = prj @ prj.T @ s1e @ mo
    return mo_rec

In [49]:
prj = np.hstack([lno_act_a[:,:0],lno_act_b[:,:]])
lno_act_b_rec = prjmo(prj,s1e,lno_act_b)
print(lno_act_b_rec.shape)
print(lno_act_b_rec-lno_act_b)

(10, 2)
[[ 3.46944695e-18  3.81639165e-17]
 [-1.38777878e-17 -1.94289029e-16]
 [-2.77555756e-17 -3.88578059e-16]
 [ 1.58818678e-22 -8.99972506e-22]
 [-2.64697796e-23 -1.72053567e-22]
 [-1.11022302e-16 -2.34187669e-17]
 [ 4.44089210e-16  1.11022302e-16]
 [ 6.24500451e-17 -2.44249065e-15]
 [-8.47032947e-22  3.04931861e-20]
 [ 6.35274710e-22  4.02340650e-21]]


In [9]:
m = np.hstack([lno_act_aa,lno_act_ba])
u,s,v = np.linalg.svd(m)
print(u.shape,s.shape,v.shape)
print(s)

(10, 10) (3,) (3, 3)
[1.41006277 1.         0.10827273]


In [13]:
def common_las(mf, lno_coeff, ncas, ncore, torr=1e-6):
    print("# Constracting cLAS that span both Alpha and Beta active space")
    s1e = mf.get_ovlp()
    lno_acta = lno_coeff[0][:,ncore[0]:ncore[0]+ncas[0]]
    lno_actb = lno_coeff[1][:,ncore[1]:ncore[1]+ncas[1]]
    lno_actaa = lno_coeff[0].T @ s1e @ lno_acta # proj to the complete
    lno_actba = lno_coeff[0].T @ s1e @ lno_actb # alpha basis for orthogonal
    clno_act = np.hstack([lno_actaa,lno_actba]) # common active lno
    u, s, _ = np.linalg.svd(clno_act)
    print(f'# Common Active Space SVD Singular values:')
    print(s)
    print(f"# cLAS projection torr: {torr}")
    for idx in range(1,clno_act.shape[1]+1):
        prj = lno_coeff[0] @ u[:,:idx]
        prj_acta = prjmo(prj,s1e,lno_actb)
        prj_actb = prjmo(prj,s1e,lno_acta)
        losa = abs(prj_acta-lno_actb).max()
        losb = abs(prj_actb-lno_acta).max()
        print(f"# cLAS projection loss: ({losa:.2e}, {losb:.2e})")
        if losa < torr and losb < torr:
            break
    print(f"# Minimum size of cLAS to span both Alpha and Beta LAS: {idx}")
    print(u.shape)
    # span{|C>} = span{|A>} U span{|B>}
    clas_coeff = lno_coeff[0] @ u[:,:idx] # in ao
    a2c = clas_coeff.T @ s1e @ lno_acta # <C|A>
    b2c = clas_coeff.T @ s1e @ lno_actb # <C|B>
    return clas_coeff, a2c, b2c

In [50]:
clas_coeff, a2c, b2c = common_las(mf, lno_coeff, ncas, ncore)
print(clas_coeff.shape)
nclas = clas_coeff.shape[1]
print(nclas)

# Constracting cLAS that span both Alpha and Beta active space
# Common Active Space SVD Singular values:
[1.41412508 1.14455766 0.83065502 0.01581954]
# cLAS projection torr: 1e-06
# cLAS projection loss: (9.08e-01, 9.94e-01)
# cLAS projection loss: (3.73e-01, 3.89e-01)
# cLAS projection loss: (1.08e-02, 1.13e-02)
# cLAS projection loss: (3.00e-16, 4.44e-16)
# Minimum size of cLAS to span both Alpha and Beta LAS: 4
(10, 10)
(10, 4)
4


In [64]:
clas_coeff[:,:4].shape

(10, 4)

In [ ]:
from pyscf import df, ao2mo, lib
chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
eri_ao = lib.einsum('lpr,lqs->prqs', chol_df, chol_df, optimize='optimal')
print("# Composing active space MO ERIs from AO ERIs")
# # FIX: find the space that just span active a and b
eri_clas = ao2mo.kernel(eri_ao,clas_coeff,compact=False)

# Composing active space MO ERIs from AO ERIs


In [57]:
from ad_afqmc import pyscf_interface
from pyscf import lib, df
chol_cut = 1e-10
nao = mol.nao
chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
chol_df = chol_df.reshape((-1, nao, nao))
print(f'# DF Tensors shape: {chol_df.shape}')
chol_df_clas = lib.einsum('pr,grs,sq->gpq',clas_coeff.T,chol_df,clas_coeff)
eri_clas = lib.einsum('lpr,lqs->prqs', chol_df_clas, chol_df_clas, optimize='optimal')
print("# Finished Composing LAS ERIs")
eri_clas = eri_clas.reshape(nclas**2,nclas**2)
print("# Decomposing MO ERIs to Cholesky vectors")
print(f"# Cholesky cutoff is: {chol_cut}")
chol_clas = pyscf_interface.modified_cholesky(eri_clas,max_error=chol_cut)
chol_clas = chol_clas.reshape((-1, nclas, nclas))
chola1 = lib.einsum('pr,grs,sq->gpq',a2c.T,chol_clas,a2c)
cholb1 = lib.einsum('pr,grs,sq->gpq',b2c.T,chol_clas,b2c)
print(f'# Alpha chol shape: {chola1.shape}')
print(f'# Betha chol shape: {cholb1.shape}')
# use DF vectors
chola2 = lib.einsum('pr,grs,sq->gpq',lno_coeff[0].T,chol_df,lno_coeff[0])
cholb2 = lib.einsum('pr,grs,sq->gpq',lno_coeff[1].T,chol_df,lno_coeff[1])
chola2 = chola2[:,ncore[0]:ncore[0]+ncas[0],ncore[0]:ncore[0]+ncas[0]]
cholb2 = cholb2[:,ncore[1]:ncore[1]+ncas[1],ncore[1]:ncore[1]+ncas[1]]
print(f'# Alpha chol shape: {chola2.shape}')
print(f'# Betha chol shape: {cholb2.shape}')

# DF Tensors shape: (172, 10, 10)
# Finished Composing LAS ERIs
# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-10
# Alpha chol shape: (12, 2, 2)
# Betha chol shape: (12, 2, 2)
# Alpha chol shape: (172, 2, 2)
# Betha chol shape: (172, 2, 2)


In [58]:
eria1 = lib.einsum('lpr,lqs->prqs',chola1,chola1)
eria2 = lib.einsum('lpr,lqs->prqs',chola2,chola2)
print(abs(eria1-eria2).max())

1.3991441338845334e-10


In [182]:
print(eri_clas.shape)

(3, 3, 3, 3)


In [183]:
from pyscf import df, ao2mo, lib
from ad_afqmc import pyscf_interface
chol_cut = 1e-5
chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
eri_ao = lib.einsum('lpr,lqs->prqs', chol_df, chol_df, optimize='optimal')
print("# Composing active space MO ERIs from AO ERIs")
# # FIX: find the space that just span active a and b
eri_clas = ao2mo.kernel(eri_ao,c,compact=False)
eri_clas = eri_clas.reshape(nclas**2,nclas**2)
print("# Decomposing MO ERIs to Cholesky vectors")
print(f"# Cholesky cutoff is: {chol_cut}")
chol_c = pyscf_interface.modified_cholesky(eri_clas,max_error=chol_cut)
chol_c = chol_c.reshape((-1, nclas, nclas))

# Composing active space MO ERIs from AO ERIs
# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-05


In [184]:
print(chol_c.shape)

(5, 3, 3)


In [187]:
# <clas|a>
a2c = c.T @ s1e @ lno_act_a
b2c = c.T @ s1e @ lno_act_b
print(a2c.shape)
print(b2c.shape)

(3, 2)
(3, 1)


In [ ]:
a2c = c.T @ s1e @ lno_act_a
b2c = c.T @ s1e @ lno_act_b
chol_a = lib.einsum('pr,grs,sq->gpq',a2c.T,chol_c,a2c)
chol_b = lib.einsum('pr,grs,sq->gpq',b2c.T,chol_c,b2c)
print(chol_a.shape, chol_b.shape)
eri_lno_a = lib.einsum('gpr,gqs->prqs',chol_a,chol_a)
eri_lno_b = lib.einsum('gpr,gqs->prqs',chol_b,chol_b)
print(eri_lno_b.shape)

(5, 2, 2) (5, 1, 1)
(1, 1, 1, 1)
